In [259]:
import pandas as pd #manipulação
import matplotlib.pyplot as plt #gráfico
import numpy as np

from sklearn import model_selection #divisão dos dados
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score

from keras import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical



from sklearn.preprocessing import LabelEncoder


K Nearest Neighbor (K Vizinhos mais próximos)
KNN = Baseado em distância. Considera proximidade entre dados na realizações de predições. Ddos similares tendem a estar concentrados na mesma região no espaço de dispersão dos dados.

In [260]:
df = pd.read_csv('wine.csv', sep=',')
display(df)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,bad
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,bad
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,bad
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,good
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,bad
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,bad
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,good
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,good
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,bad


In [261]:
#Convertendo os dados para um array
array = df.values
array

array([[7.4, 0.7, 0.0, ..., 0.56, 9.4, 'bad'],
       [7.8, 0.88, 0.0, ..., 0.68, 9.8, 'bad'],
       [7.8, 0.76, 0.04, ..., 0.65, 9.8, 'bad'],
       ...,
       [6.3, 0.51, 0.13, ..., 0.75, 11.0, 'good'],
       [5.9, 0.645, 0.12, ..., 0.71, 10.2, 'bad'],
       [6.0, 0.31, 0.47, ..., 0.66, 11.0, 'good']], dtype=object)

In [262]:
#Array com os inputs
#Pegando todas as linhas (:, da linha 0 até a antepenultima)
X = array[:, 0:11]

#Array com a definição (classe)
Y = array[:, 11]

In [263]:
dummies = pd.get_dummies(df.quality)
df_2 = pd.concat((dummies, df), axis=1)
df_2 = df_2.drop(['quality'], axis=1)
df_2 = df_2.drop(['bad'], axis=1)
df_new = df_2.rename(columns={"good": "quality"}) # bad = 1, good = 0
df = df_new
df

,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,0,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,0,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,1,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,0,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1595,1,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,1,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1597,0,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2


In [264]:
#Tamanho do treino (20%)
test_group_size = 0.2
seed = 6

X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size = test_group_size)

In [265]:
from sklearn.neighbors import KNeighborsClassifier

#Define o modelo KNN com vizinhos = 3
model = KNeighborsClassifier(n_neighbors = 3)

#Treina o algortimo de treinamento
clf = model.fit(X_train, Y_train)

#Predição com base o conjunto de validação
predictions = model.predict(X_validation)

In [266]:
#Calcula a acurácia
#Compara classe real com as predições
print(accuracy_score(Y_validation, predictions))

0.703125


In [271]:
auc = roc_auc_score(Y_train, predictions)
print('ROC AUC: %f' % auc)

ValueError: ignored

In [267]:
#Mattiz de confusão
print(confusion_matrix(Y_validation, predictions))
# VP | FP
# FN | VN

[[100  52]
 [ 43 125]]


In [268]:
precision = precision_score(Y_train, predictions)
print('Precision: %f' % precision)

ValueError: ignored